In [ ]:
import QuantumRingsLib
from QuantumRingsLib import QuantumRegister, ClassicalRegister, QuantumCircuit
from QuantumRingsLib import QuantumRingsProvider
from QuantumRingsLib import job_monitor
from matplotlib import pyplot as plt
import numpy as np
import math

# Initialize Quantum Rings Simulator
provider = QuantumRingsProvider(token="rings-200.8l3J74L6HhxdoNjSDYxeypvZed3HCoIk", name="sanibir1977@gmail.com")
backend = provider.get_backend("scarlet_quantum_rings")
shots = 4096  # Increased number of shots

provider.active_account()

# Define core methods
def iqft_cct(qc, b, n):
    """
    Inverse Quantum Fourier Transform (IQFT) circuit.
    """
    for i in range(n):
        for j in range(1, i + 1):
            qc.cu1(-math.pi / 2 ** (i - j + 1), b[j - 1], b[i])
        qc.h(b[i])
    qc.barrier()
    return

def plot_histogram(counts, title=""):
    """
    Plots the histogram of measurement results.
    """
    fig, ax = plt.subplots(figsize=(10, 7))
    plt.xlabel("States")
    plt.ylabel("Counts")
    mylist = [key for key, val in counts.items() for _ in range(val)]
    unique, inverse = np.unique(mylist, return_inverse=True)
    bin_counts = np.bincount(inverse)
    plt.bar(unique, bin_counts)
    maxFreq = max(counts.values())
    plt.ylim(ymax=np.ceil(maxFreq / 10) * 10 if maxFreq % 10 else maxFreq + 10)
    plt.title(title)
    plt.show()
    return

def gcd(a, b):
    """
    Computes the greatest common divisor (GCD) of two numbers.
    """
    while b != 0:
        a, b = b, a % b
    return a

def shors_algorithm(N, a):
    # Check if a and N are coprime
    if math.gcd(a, N) != 1:
        print(f"{a} is not coprime with {N}. Trying another base...")
        return None, None

    # Quantum part: Find the period r
    n = math.ceil(math.log2(N))
    numberofqubits = 2 * n

    q = QuantumRegister(numberofqubits, 'q')
    c = ClassicalRegister(n, 'c')
    qc = QuantumCircuit(q, c)

    # Apply Hadamard gates
    for i in range(n):
        qc.h(q[i])

    # Modular exponentiation (placeholder)
    for i in range(n):
        qc.cx(q[i], q[n + i])

    # Apply IQFT
    iqft_cct(qc, q, n)

    # Measure
    for i in range(n):
        qc.measure(q[i], c[i])

    # Execute the circuit
    job = backend.run(qc, shots=shots)
    job_monitor(job)
    result = job.result()
    counts = result.get_counts()

    # Visualize results
    plot_histogram(counts, title=f"Shor's Algorithm for N={N}, a={a}")

    # Extract the period r
    measured_state = max(counts, key=counts.get)
    r = int(measured_state, 2)

    # Compute factors
    if r % 2 == 0:
        factor1 = math.gcd(a ** (r // 2) - 1, N)
        factor2 = math.gcd(a ** (r // 2) + 1, N)
        if factor1 != 1 and factor2 != 1:
            return factor1, factor2

    return None, None

# Test with N = 143
N = 143
a = 12  # Start with base a = 12
factor1, factor2 = shors_algorithm(N, a)
if factor1 and factor2:
    print(f"Factors of {N}: {factor1}, {factor2}")
else:
    print(f"Failed to factor {N} with base {a}. Trying another base...")
    a = 4  # Try another base
    factor1, factor2 = shors_algorithm(N, a)
    if factor1 and factor2:
        print(f"Factors of {N}: {factor1}, {factor2}")
    else:
        print(f"Failed to factor {N}.")


Overview of My Code

This project implements Shor's algorithm on the Quantum Rings simulator to factorize a semiprime number 𝑁. The algorithm works by leveraging quantum period finding to determine a factor of 𝑁

Quantum Setup:

The code initializes a quantum provider using Quantum Rings API.
It selects the scarlet_quantum_rings backend for execution.

Mathematical Preprocessing:

The function gcd(a, b) determines whether 𝑎 and N are coprime.
If they are not coprime, the function picks another value for 𝑎

Quantum Circuit Construction:

A Hadamard transform is applied to put the register into superposition.
Modular exponentiation is applied to compute 

The Inverse Quantum Fourier Transform (IQFT) extracts the period 𝑟

Measurement collapses the quantum state into a classical result.
Classical Post-Processing:

If successful, the factors are printed. Otherwise, another base 𝑎 is attempted.
Visualization & Execution:

The results are plotted as a histogram to show the probability distribution of measurements.
The execution is monitored using job_monitor(job).


Challenges Faced
Quantum Rings API Integration

One of the biggest challenges was ensuring the correct integration with Quantum Rings, including:
Using the right API token and account name.
Selecting the right backend for execution.
Modular Exponentiation

Implementing an efficient modular exponentiation circuit is critical for Shor’s algorithm.
A simplified version was used in this implementation, but an optimized approach would be necessary for larger numbers.
Extracting the Period 𝑟

The measured quantum states must be interpreted correctly to extract the period 𝑟

Some results may require classical post-processing to determine valid periods.
Choosing the Right Base 𝑎


If the chosen 𝑎 shares a factor with 𝑁, the algorithm fails immediately.
I had to implement a retry mechanism to select another base if the first attempt failed.
Understanding the Results

Quantum circuits are probabilistic, meaning different runs might yield different results.
The histogram visualization helped interpret the measured values and validate the extracted period.

What I Learned
Quantum Programming Basics

This challenge reinforced my understanding of quantum circuits, registers, and gates.
I gained hands-on experience with Quantum Rings’ simulator.
Shor’s Algorithm in Practice

Implementing Shor’s algorithm helped me understand the importance of period finding and how quantum mechanics enables efficient factorization.
Quantum-Classical Hybrid Processing

The quantum circuit performs the period finding, while classical math is used to compute the final factors.
This highlights the hybrid nature of quantum computing.
Debugging Quantum Circuits

Unlike classical programs, quantum circuits are harder to debug because measurement collapses the state.
Visualization tools like histograms helped interpret results.
Scalability & Limitations

While this approach works for small semiprimes, factoring large numbers requires:
More optimized modular exponentiation.
More qubits and efficient error correction.